In [6]:
import pandas as pd

In [9]:
path = '../data/jumble_data.json'

df = pd.read_json(path_or_buf=path, lines=True)
df.sort_values("value_date", inplace=True)

In [29]:
df_cues = df[df['jumbled'].apply(lambda x: len(x) > 17)].copy()
tt = df_cues.set_index("value_date")
tt.index = pd.to_datetime(tt.index)
tt = tt.resample('D').first()

In [37]:
365*3

1095

In [31]:
tt[tt['jumbled'].isnull()]

,jumbled,unjumbled
value_date,,
2022-11-22,None,None
2023-09-23,None,None
2023-10-01,None,None
